  <picture>
  <source srcset="https://raw.githubusercontent.com/Archfx/RACKETutes/main/images/racketutes-w.svg" media="(prefers-color-scheme: dark)">
    <img src="https://raw.githubusercontent.com/Archfx/RACKETutes/main/images/racketutes.svg">
  </picture>

Rosette101
===

The previous post was about getting familiar with Racket. To make things more interesting we have used Jupyter notebook interactive environment. This time was are going to use Rosette in the same environment.

Rosette is a solver-aided programming language. To be specific popular [Z3](https://github.com/Z3Prover/z3) is behind the curtains of Rosette. With the power of Z3, Rosette programs can understand and handle symbolic values, assertions, assumptions, and queries. Which is a vital part of verifying firmware and hardware designs.

Note: Rosette's symbolic evaluation engine operates on the program states. Instead of operating on one path at a time, it keeps track of all assumptions and assertions in the path at the beginning.  This is done through the `verification` condition (VC)`.  During the tutorial, we will be switching contexts between different examples and problems. Since we are using the same kernel to run different examples, we need to clean the VC when we switch the context.

In [1]:
#lang iracket/lang #:require rosette/safe 

Symbolic Values
---

When it comes to validating designs and implementations, one thing is we can't visualize all the internal variables. These variables are sometimes constrained by different conditions. With Rosette, we can keep these variables as unknown variable variables which refers to the technical term of ```symbolic variables```. These variables are symbolically evaluated on the constraints that act on them. 

In [2]:
(define-symbolic b boolean?)
(displayln b)

(displayln (boolean? b))
(displayln (integer? b))
(displayln (vector b 1))
(displayln (not b))
(displayln (boolean? (not b)))

b
#t
#f
#(b 1)
(! b)
#t


Assertions and Assumptions
---

As the name suggests, Rosette's assertions work exactly similarly to assertions used in other languages. If the assertion is failed, it will terminate the exection with an exception.

In [21]:
(assert #t)
(assert #f)

[assert] failed
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/core/exn.rkt:59:11: body of top-level


In [22]:
(clear-vc!)

In [23]:
(define-symbolic i j integer?)
(assume (> j 0))     ; Add the assumption (> j 0) to the VC.
(vc-assumes (vc))

(< 0 j)

In [24]:
(assert (< (- i j) i))
(vc-asserts (vc))    ; The assertions must hold when the assumptions hold.

(display (vc))

#(struct:vc (< 0 j) (|| (! (< 0 j)) (< (+ i (- j)) i)))

In [6]:
; int32? is a shorthand for the type (bitvector 32).
(define int32? (bitvector 32))

; int32 takes as input an integer literal and returns
; the corresponding 32-bit bitvector value.
(define (int32 i)
  (bv i int32?))


In [ ]:
(clear-vc!)

Solver-Aided Queries
---

In [7]:
(define (poly x)
 (+ (* x x x x) (* 6 x x x) (* 11 x x) (* 6 x)))
 
(define (factored x)
 (* x (+ x 1) (+ x 2) (+ x 2)))
 
(define (same p f x)
 (assert (= (p x) (f x))))

In [10]:
(same poly factored 0)
(same poly factored -1)
(same poly factored -2)

In [ ]:
(clear-vc!)

Verification
---

Program verification solves the problem of verifying the program for all the legal inputs. The brute force approach would be to iterate through all the values one by one. Although this sounds promising at capturing any bug if exists, in the case of a 32-bit adder circuit, input space would be $2^{64}$ which is a large number.
Instead of going through the brute force path, Rosette delegates this task to `z3 constraint solver` with the help of `verify` query.


In [11]:
(define-symbolic i integer?)
(define cex (verify (same poly factored i)))


In [15]:
(evaluate i cex)
(poly -6)
(factored -6)
; (same poly factored -6)

480

In [51]:
(clear-vc!)


Synthesis
----

The solver can not only find failure-inducing inputs and localize faults, it can also synthesize repairs for buggy expressions. To repair a program, we first replace each buggy expression with a syntactic "hole." A program with holes is called a sketch. The solver completes a sketch by filling its holes with expressions, in such a way that all assertions in the resulting program pass on all inputs.

The following code snippet shows the sketch for our buggy factored procedure. We obtained it by replacing the constants in the minimal core with (??) holes, which are filled with numerical constants.4

In [52]:

(require rosette/lib/synthax)

(define (poly x)
 (+ (* x x x x) (* 6 x x x) (* 11 x x) (* 6 x)))
 
(define (factored x)
 (* (+ x (??)) (+ x 1) (+ x 2) (+ x 3)))
 
(define (same p f x)
 (assert (= (p x) (f x))))


In [53]:
(define-symbolic i integer?)
(define binding
    (synthesize #:forall (list i)
                #:guarantee (same poly factored i)))
(print-forms binding)

open-input-file: contract violation
  expected: path-string?
  given: 'eval
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/lib/util/syntax.rkt:32:0: read-module
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/lib/synthax.rkt:243:0: body of top-level
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/lib/synthax.rkt:296:0: print-forms


In [45]:
(clear-vc!)

Angelic Execution
----

In [29]:
(define-symbolic x y integer?)
(define sol
    (solve (begin (assert (not (= x y)))
                  (assert (< (abs x) 10))
                  (assert (< (abs y) 10))
                  (assert (not (= (poly x) 0)))
                  (assert (= (poly x) (poly y))))))
(evaluate x sol)

(evaluate y sol)

(evaluate (poly x) sol)

(evaluate (poly y) sol)

24

In [30]:
(clear-vc!)

#### Lets Do an experiment
For this experiment, we generate two implementations for 32-bit adders which utilize Ripple-carry adder and Carry lookahead adder using ChatGPT. Our task is to verify these two implementations. 

Ripple-carry adder implementation is as below,

In [12]:
#lang iracket/lang #:require racket/base

In [2]:
; add-32: Number Number -> Number
; Purpose: Add two 32-bit numbers and return the result
; Pre-conditions: The two input numbers must be 32-bit unsigned integers (0 <= x, y < 2^32)
;Post-conditions: Returns the sum of the two input numbers as a 32-bit unsigned integer (0 <= result < 2^32)

(define (add-32 x y)
  (let ((result 0)
        (carry 0))
    (for ([i (in-range 32)])
      (let ((bit-x (bitwise-and x (expt 2 i)))
            (bit-y (bitwise-and y (expt 2 i))))
        (let ((sum (bitwise-xor bit-x bit-y carry)))
          (set! result (bitwise-ior result sum))
          (set! carry (bitwise-and (bitwise-or (bitwise-and bit-x bit-y)
                                               (bitwise-and sum carry))
                                   (expt 2 (add1 i)))))))
    result))


Test for several test cases

In [3]:
(add-32 10 15)

bitwise-or: undefined;
 cannot reference an identifier before its definition
  in module: top-level
  context...:
   body of top-level
   eval:6:0: add-32


Carry lookahead adder Implementation is as below,

In [4]:
;; add-32-cla: Number Number -> Number
;; Purpose: Add two 32-bit numbers using a carry lookahead adder and return the result
;; Pre-conditions: The two input numbers must be 32-bit unsigned integers (0 <= x, y < 2^32)
;; Post-conditions: Returns the sum of the two input numbers as a 32-bit unsigned integer (0 <= result < 2^32)


(define (add-32-cla x y)
  (let* ((g (make-vector 32 0))
         (p (make-vector 32 0))
         (c (make-vector 32 0))
         (cin 0)
         (cout (vector-ref c 31)))
    ;; Generate and propagate bits for all positions
    (for ([i (in-range 32)])
      (vector-set! g i (bitwise-and (vector-ref x i) (vector-ref y i)))
      (vector-set! p i (bitwise-xor (vector-ref x i) (vector-ref y i))))
    ;; Compute carry bits for each position
    (for ([i (in-range 32)])
      (vector-set! c i (vector-ref p i)))
    (for ([i (in-range 1 32)])
      (let ((g-and-c (bitwise-and (vector-ref g i) (vector-ref c (- i 1)))))
        (vector-set! c i (bitwise-or (vector-ref c i) g-and-c))))
    ;; Compute sum bits for each position
    (let ((result (make-vector 32 0)))
      (for ([i (in-range 32)])
        (let ((s (bitwise-xor (vector-ref p i) cin)))
          (vector-set! result i s)
          (set! cin (if (vector-ref c i) 1 0))))
      (bitwise-bignum-to-number result))))




Test with several test cases

In [11]:
 (add-32-cla (vector 0 1 0 1 0 1 0 1
                                 0 1 0 1 0 1 0 1
                                 0 1 0 1 0 1 0 1
                                 0 1 0 1 0 1 0 1)
                              (vector 1 0 1 0 1 0 1 0
                                 1 0 1 0 1 0 1 0
                                 1 0 1 0 1 0 1 0
                                 1 0 1 0 1 0 1 0))

bitwise-or: undefined;
 cannot reference an identifier before its definition
  in module: top-level
  context...:
   body of top-level
   eval:7:0: add-32-cla


In [28]:
#lang iracket/lang #:require rosette/safe

In [29]:


(define (quadratic-roots a b c)
  (define delta (- (* b b) (* 4 a c)))
  (if (< delta 0)
      #f
      (let* ([sqrt-delta (sqrt delta)]
             [denominator (* 2 a)])
        (list (/ (- (- b) sqrt-delta) denominator)
              (/ (- (- b) (- sqrt-delta)) denominator)))))

In [30]:

(define-syntax-rule (check-equal actual expected)
  (if (equal? actual expected)
      (void)
      (error 'check-equal "expected: ~s, got: ~s" expected actual)))

(check-equal (quadratic-roots 1 -5 6) '(2 3))
(check-equal (quadratic-roots 1 2 1) '(-1 -1))
(check-equal (quadratic-roots 1 -2 1) '(1 1))
(check-equal (quadratic-roots 1 1 1) #f)

sqrt: undefined;
 cannot reference an identifier before its definition
  in module: top-level
  context...:
   body of top-level
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/form/control.rkt:16:25
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


### Some Problems from Internet


Finding an integer whose absolute value is X with Racket

In [1]:
#lang iracket/lang #:require rosette/safe 

(define (abs x)
    (if (< x 0) (- x) x))

In [2]:
; define a symbolic variable y

(define-symbolic y integer?)

In [3]:
; Solve a constraint saying |y| = 5.
(solve
  (assert (= (abs y) 5)))

(model
 [y -5])

In [31]:
(clear-vc!)


Now let us look at some sample questions from CAV 2019 which was developed by [Emina](https://github.com/emina)




#### Q1. Warmup

Suppose that you are an app developer for a super low-power chip that achieves peak energy efficiency on programs with few or no branches. To build apps faster, you hire Ben Bitdiddle to create a library of efficient primitives for this chip, starting with a branch-free function that computes the absolute value of a 32-bit bitvector. 

In [1]:
#lang iracket/lang #:require rosette

In [2]:
(define (abs-spec x)
  (if (bvslt x (bv 0 32))
      (bvneg x)
      x))

Ben comes up with the following implementation that works on his test suite:

In [3]:
(define (abs-impl x) 
  (let* ([o1 (bvashr x (bv 31 32))]
         [o2 (bvadd x o1)]
         [o3 (bvsub o2 o1)])
    o3))

; zero, positive, negative ...
(assert (equal? (abs-impl (bv #x00000000 32)) 
                  (abs-spec (bv #x00000000 32))))
(assert (equal? (abs-impl (bv #x00000003 32)) 
                  (abs-spec (bv #x00000003 32))))
(assert (equal? (abs-impl (bv #x80000000 32)) 
                  (abs-spec (bv #x80000000 32))))

Help review Ben’s code by answering the following questions. If a question requires you to modify his implementation in any way, do so on a fresh copy with a new name such as abs-impl-N where N is the question number.

Does Ben’s implementation work on all 32-bit inputs? Use the verify query to check.

In [5]:
; Let's define a generic 32bit vector
(define-symbolic x (bitvector 32))

; Using Rosette verify query
(define check
  (verify
   (assert(equal? (abs-spec x) (abs-impl x)))))

(evaluate check x)

(model
 [x (bv #x8404c7c0 32)])

Apparently, we have a counter-example where implementation does not satisfy the specification.

The verifier will reveal that Ben’s implementation is buggy. Use debug to localize the fault found by the verifier (and remember to save your work to a file before invoking debug).

Use synthesize to fix Ben’s implementation as suggested by the debugger. You can assume that if Ben has made an error, it is always of the same kind: using an arithmetic operator (such as addition or subtraction) instead of a bitwise operator that takes the same arguments. It is easiest to sketch this knowledge using the choose construct.

In [10]:
(require rosette/lib/synthax)

(define (abs-impl-3 x) 
  (let* ([o1 (bvashr x (bv 31 32))]
         [o2 ((choose bvadd bvand bvor bvxor bvshl bvlshr bvashr) x o1)]
         [o3 ((choose bvsub bvand bvor bvxor bvshl bvlshr bvashr) o2 o1)])
    o3))


In [12]:
(synthesize
  #:forall x
  #:guarantee (assert (equal? (abs-spec x) (abs-impl-3 x))))

(model
 [0$choose:eval:5:15 #t]
 [1$choose:eval:5:15 #f]
 [2$choose:eval:5:15 #f]
 [3$choose:eval:5:15 #f]
 [4$choose:eval:5:15 #f]
 [5$choose:eval:5:15 #f]
 [0$choose:eval:6:15 #f]
 [1$choose:eval:6:15 #f]
 [2$choose:eval:6:15 #f]
 [3$choose:eval:6:15 #t])

#### Q2. Majority Voting Algorithm

The [Boyer–Moore majority vote algorithm](https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore_majority_vote_algorithm) is a beautiful and tricky procedure for finding a majority element in a list, using linear time and constant space.

Complete the following Rosette implementation of Boyer-Moore and verify its correctness for any list of N arbitrary integers.

In [2]:
#lang iracket/lang #:require rosette

In [32]:
(define (boyer-moore xs)
  (define m (car xs))  ; car returns the first element of an array
  (define i 0)
  (for ([x xs])
    (if (= i 0) (and (set! m x) (set! i 1))
          (if (= m x) (set! i (+ i 1)) 
          (set! i (- i 1)))
          ))
  m)

Test for several test cases in traditional way

In [3]:
(display "case 1 :")(displayln (= 1 (boyer-moore `(1 1 2 4 1 1))))
(display "case 2 :")(displayln (= 4 (boyer-moore `(8 6 4 1 2 4 1 1 8 4 8 4 4))))

case 1 :#t
case 2 :#t


In [33]:
;Check the algorithm for a list of n arbitrary integers.
(define (check n [bw #f])
  (current-bitwidth bw)
  (define-symbolic* xs integer? [n])
  (define-symbolic* m integer?)
  (verify 
  #:assume ; there is a majority value ...
   (assert (> (count (curry = m) xs)
              (quotient n 2)))
   #:guarantee ; algorithm works
   (assert (= m (boyer-moore xs)))))

(time (check 10)) 



eval:6:2: verify: use does not match pattern: (verify expr)
  in: (verify #:assume (assert (> (count (curry = m) xs) (quotient n 2))) #:guarantee (assert (= m (boyer-moore xs))))
  location...:
   eval:6:2
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/form/module.rkt:33:2: try-next
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


In [34]:
; Check the algorithm for a list of n arbitrary integers.
(define (check2 n [bw #f])
  (current-bitwidth bw)
  (define-symbolic* xs integer? [n])
  (define-symbolic* m integer?)
  (verify
   (when ; there is a majority value ...
       (> (count (curry = m) xs)
          (quotient n 2))
     (assert (= m (boyer-moore xs)))))) ; algorithm works

(time (check2 1)) 


application: not a procedure;
 expected a procedure that can be applied to arguments
  given: 1
  context...:
   body of top-level
   eval:2:0: check2


In [35]:
(clear-vc!)


#### 3. Sudoku

In [57]:
#lang iracket/lang #:require rosette

In [59]:
(define (choice type) 
  (define-symbolic* c type) 
  c)

(define (more-or-less-1 x)
  (if (choice boolean?)
      (+ x 1)
      (- x 1)))

(solve (assert (= 0 (more-or-less-1 1)))) ; #f
(solve (assert (= 2 (more-or-less-1 1)))) ; #t

(model
 [a #t]
 [b #t]
 [c$6 #t])

In [60]:
(assert #f)

[assert] failed
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/core/exn.rkt:59:11: body of top-level


In [61]:
(define (poly x)
 (+ (* x x x x) (* 6 x x x) (* 11 x x) (* 6 x)))
 
(define (factored x)
 (* x (+ x 1) (+ x 2) (+ x 2)))
 
(define (same p f x)
 (assert (= (p x) (f x))))

In [66]:
(same poly factored 0)

In [63]:
(same poly factored -1)

In [64]:
(same poly factored -2)

In [2]:
#lang iracket/lang #:require rosette/safe 

(define-symbolic l h int32?)

(define cex (verify (check-mid bvmid l h)))


int32?: undefined;
 cannot reference an identifier before its definition
  in module: top-level
  context...:
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


In [6]:
#lang iracket/lang #:require yosys
; SMT-LIBv2 description generated by Yosys 0.23+25 (git sha1 13e4f343b, clang 10.0.0-4ubuntu1 -fPIC -Os)
; yosys-smt2-stdt
; yosys-smt2-module adder
(declare-datatype |adder_s| ((|adder_mk|
  (|adder_is| Bool)
  (|adder#0| (_ BitVec 32)) ; \a
  (|adder#1| (_ BitVec 32)) ; \b
)))
; yosys-smt2-input a 32
; yosys-smt2-witness {"offset": 0, "path": ["\\a"], "smtname": "a", "smtoffset": 0, "type": "input", "width": 32}
(define-fun |adder_n a| ((state |adder_s|)) (_ BitVec 32) (|adder#0| state))
; yosys-smt2-input b 32
; yosys-smt2-witness {"offset": 0, "path": ["\\b"], "smtname": "b", "smtoffset": 0, "type": "input", "width": 32}
(define-fun |adder_n b| ((state |adder_s|)) (_ BitVec 32) (|adder#1| state))
(define-fun |adder#2| ((state |adder_s|)) (_ BitVec 32) (bvadd (|adder#0| state) (|adder#1| state))) ; \c
; yosys-smt2-output c 32
(define-fun |adder_n c| ((state |adder_s|)) (_ BitVec 32) (|adder#2| state))
(define-fun |adder_a| ((state |adder_s|)) Bool true)
(define-fun |adder_u| ((state |adder_s|)) Bool true)
(define-fun |adder_i| ((state |adder_s|)) Bool true)
(define-fun |adder_h| ((state |adder_s|)) Bool true)
(define-fun |adder_t| ((state |adder_s|) (next_state |adder_s|)) Bool true) ; end of module adder
; yosys-smt2-topmod adder
; end of yosys output


#%top-interaction: unbound identifier;
 also, no #%app syntax transformer is bound
  at: #%top-interaction
  in: (#%top-interaction declare-datatype adder_s ((adder_mk (adder_is Bool) (adder#0 (_ BitVec 32)) (adder#1 (_ BitVec 32)))))
  context...:
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


In [63]:
(add_n 1)

#%top-interaction: unbound identifier;
 also, no #%app syntax transformer is bound
  at: #%top-interaction
  in: (#%top-interaction add_n 1)


In [21]:
(clear-vc!)

In [4]:
(define (adder_spec a b)
  (+ a b))

In [1]:
(adder_spec 10 15)

adder_spec: undefined;
 cannot reference an identifier before its definition
  in module: top-level
  context...:
   body of top-level


> Hello this is a test message

### Some codes from chatGPT

In [3]:
(define arr '(4 2 7 1 5))
(define sorted-arr (sort arr <))
(displayln sorted-arr)

(1 2 4 5 7)


In [4]:
(require 2htdp/image)

(define (make-chessboard size)
  (let* ((square-size (/ size 8))
         (odd-square (square-size 255 204 153))
         (even-square (square-size 204 153 102)))
    (beside
     (above-map (lambda (row)
                  (below-map (lambda (col)
                               (if (even? (+ row col))
                                   even-square
                                   odd-square))
                             (range 8)))
                (range 8)))))

(define chessboard (make-chessboard 400))

(draw chessboard)

application: not a procedure;
 expected a procedure that can be applied to arguments
  given: 50
  context...:
   body of top-level
   eval:3:0: make-chessboard
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


In [13]:

 (require 2htdp/image)

In [6]:
(circle 30 "outline" "red")

(object:image% #f #f #f #f 1 0 (object:image-snipclass% #f "((lib \"image-core.ss\" \"mrlib\") (lib \"image-core-wxme.rkt\" \"mrlib\"))" 1) #0=(object:style% (object:color% 0 0 0 1.0 #f) (object:color% 255 255 255 1.0 #f) (object:font% #f 'aligned '#(12.0 default normal normal #f default #f aligned) #f #f #f 'default 12.0 #f 'default 'normal 'normal 1 '#(1 12.0 normal normal #f default #f aligned)) 'bottom 0.0 0.0 0.0 0.0 (object:style-list% '#<hash> #0# '(#0#) 1 #<procedure:add-member>) "Basic" #f #f (object:style-delta% 'default #f 0 12 'normal 'base 'normal 'base 'default 'base #f #t #f #t #f #f 'bottom 'base (object:mult-color% 0 0 0) (object:mult-color% 0 0 0) (object:add-color% 0 0 0) (object:add-color% 255 255 255)) #t (object:pen% '#(0 0 0 1.0 0 solid round round) #t 0 #f (object:color% 0 0 0 1.0 #t) 'round 'round 'solid 0) (object:brush% '#(255 255 255 1.0 solid) (object:color% 255 255 255 1.0 #t) 'solid #t 0 #f #f #f #f) 0 '()) (translate 30 30 (ellipse 60 60 0 'outline "red" #f)) (bb 60 60 60) #f #f #f #f #t)

In [11]:
(add-line
    (rectangle 100 100 "solid" "darkolivegreen")
    25 25 75 75
    (make-pen "goldenrod" 30 "solid" "round" "round"))

(object:image% #f #f #f #f 1 0 (object:image-snipclass% #f "((lib \"image-core.ss\" \"mrlib\") (lib \"image-core-wxme.rkt\" \"mrlib\"))" 1) #0=(object:style% (object:color% 0 0 0 1.0 #f) (object:color% 255 255 255 1.0 #f) (object:font% #f 'aligned '#(12.0 default normal normal #f default #f aligned) #f #f #f 'default 12.0 #f 'default 'normal 'normal 1 '#(1 12.0 normal normal #f default #f aligned)) 'bottom 0.0 0.0 0.0 0.0 (object:style-list% '#<hash> #0# '(#0#) 1 #<procedure:add-member>) "Basic" #f #f (object:style-delta% 'default #f 0 12 'normal 'base 'normal 'base 'default 'base #f #t #f #t #f #f 'bottom 'base (object:mult-color% 0 0 0) (object:mult-color% 0 0 0) (object:add-color% 0 0 0) (object:add-color% 255 255 255)) #t (object:pen% '#(0 0 0 1.0 0 solid round round) #t 0 #f (object:color% 0 0 0 1.0 #t) 'round 'round 'solid 0) (object:brush% '#(255 255 255 1.0 solid) (object:color% 255 255 255 1.0 #t) 'solid #t 0 #f #f #f #f) 0 '()) (translate 0 0 (overlay (line-segment (point 25 25) (point 75 75) (pen "goldenrod" 30 'solid 'round 'round)) (translate 0 0 (polygon (list (pulled-point 0 0 0 0 0 0) (pulled-point 0 0 100 0 0 0) (pulled-point 0 0 100 100 0 0) (pulled-point 0 0 0 100 0 0)) 255 "darkolivegreen")))) (bb 100 100 100) #f #f #f #f #t)

In [16]:

 (require lang/posn)

(underlay
   (rectangle 80 80 "solid" "mediumseagreen")
   (polygon
    (list (make-posn 0 0)
          (make-posn 50 0)
          (make-posn 0 50)
          (make-posn 50 50))
    "outline"
    (make-pen "darkslategray" 10 "solid" "round" "round")))


(object:image% #f #f #f #f 1 0 (object:image-snipclass% #f "((lib \"image-core.ss\" \"mrlib\") (lib \"image-core-wxme.rkt\" \"mrlib\"))" 1) #0=(object:style% (object:color% 0 0 0 1.0 #f) (object:color% 255 255 255 1.0 #f) (object:font% #f 'aligned '#(12.0 default normal normal #f default #f aligned) #f #f #f 'default 12.0 #f 'default 'normal 'normal 1 '#(1 12.0 normal normal #f default #f aligned)) 'bottom 0.0 0.0 0.0 0.0 (object:style-list% '#<hash> #0# '(#0#) 1 #<procedure:add-member>) "Basic" #f #f (object:style-delta% 'default #f 0 12 'normal 'base 'normal 'base 'default 'base #f #t #f #t #f #f 'bottom 'base (object:mult-color% 0 0 0) (object:mult-color% 0 0 0) (object:add-color% 0 0 0) (object:add-color% 255 255 255)) #t (object:pen% '#(0 0 0 1.0 0 solid round round) #t 0 #f (object:color% 0 0 0 1.0 #t) 'round 'round 'solid 0) (object:brush% '#(255 255 255 1.0 solid) (object:color% 255 255 255 1.0 #t) 'solid #t 0 #f #f #f #f) 0 '()) (overlay (translate 15 15 (translate 0 0 (polygon (list (pulled-point 0 0 0 0 0 0) (pulled-point 0 0 50 0 0 0) (pulled-point 0 0 0 50 0 0) (pulled-point 0 0 50 50 0 0)) 'outline (pen "darkslategray" 10 'solid 'round 'round)))) (translate 0 0 (translate 0 0 (polygon (list (pulled-point 0 0 0 0 0 0) (pulled-point 0 0 80 0 0 0) (pulled-point 0 0 80 80 0 0) (pulled-point 0 0 0 80 0 0)) 255 "mediumseagreen")))) (bb 80 80 80) #f #f #f #f #t)

In [1]:
#lang iracket/lang #:require racket/base 

In [8]:

(require 2htdp/image)
(require 2htdp/universe)

(define padding 0.1)
(define size 400)
(define width (- size (* size padding)))
(define height (- size (* size padding)))
(define lambda-width (/ width 2))
(define lambda-height (/ height 2))

; (define black-square (rectangle lambda-width lambda-height "solid" "black"))
; (define rotated-black-square (rotate 45 black-square))
; (define white-background (rectangle width height "solid" "white"))

(rectangle lambda-width lambda-height "solid" "black")
; (define racket-logo
  ; (overlay/align "center"
  ;   white-background
  ;   (overlay (scale (* 0.9) (* 0.1) black-square)
  ;            (scale (* 0.9) (* 0.1) rotated-black-square)));)


; (big-bang 0 (on-draw (lambda (_) racket-logo)))


instantiate-linklet: mismatch;
 reference to a variable that is uninitialized;
 possibly, bytecode file needs re-compile because dependencies changed
  name: get-double-click-time
  exporting instance: "/usr/share/racket/pkgs/gui-lib/mred/private/wx/platform.rkt"
  importing instance: "/usr/share/racket/pkgs/gui-lib/mred/private/wxme/wx.rkt"
  context...:
   body of "/usr/share/racket/pkgs/gui-lib/mred/private/wxme/wx.rkt"
   body of top-level
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


In [8]:
(pulled-regular-polygon 100 3 1.8 30 "solid" "blue")



    

(object:image% #f #f #f #f 1 0 (object:image-snipclass% #f "((lib \"image-core.ss\" \"mrlib\") (lib \"image-core-wxme.rkt\" \"mrlib\"))" 1) #0=(object:style% (object:color% 0 0 0 1.0 #f) (object:color% 255 255 255 1.0 #f) (object:font% #f 'aligned '#(12.0 default normal normal #f default #f aligned) #f #f #f 'default 12.0 #f 'default 'normal 'normal 1 '#(1 12.0 normal normal #f default #f aligned)) 'bottom 0.0 0.0 0.0 0.0 (object:style-list% '#<hash> #0# '(#0#) 1 #<procedure:add-member>) "Basic" #f #f (object:style-delta% 'default #f 0 12 'normal 'base 'normal 'base 'default 'base #f #t #f #t #f #f 'bottom 'base (object:mult-color% 0 0 0) (object:mult-color% 0 0 0) (object:add-color% 0 0 0) (object:add-color% 255 255 255)) #t (object:pen% '#(0 0 0 1.0 0 solid round round) #t 0 #f (object:color% 0 0 0 1.0 #t) 'round 'round 'solid 0) (object:brush% '#(255 255 255 1.0 solid) (object:color% 255 255 255 1.0 #t) 'solid #t 0 #f #f #f #f) 0 '()) (translate -0.0 86.60254037844388 (polygon (list (pulled-point 1.8 30 0 0 1.8 -30) (pulled-point 1.8 30 100 0 1.8 -30) (pulled-point 1.8 30 50.00000000000002 -86.60254037844388 1.8 -30)) 255 "blue")) (bb 100.0 86.60254037844388 86.60254037844388) #f #f #f #f #t)